In [1]:
import json
import pandas as pd
import numpy as np
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
from math import pi
from pprint import pprint
from konlpy.tag import Okt
from collections import Counter
from sklearn.manifold import TSNE
from future.utils import iteritems
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

In [2]:
import os

def read_data(filename):
    with open(filename, 'r') as f:
        data = f.read()
        # remove_quotes
        data = data.replace("‘", " ")
        data = data.replace("’", " ")
        data = data.replace("“", " ")
        data = data.replace("”", " ")
        data = data.replace("`", " ")
        data = data.replace("\'", " ")
        data = data.replace("\"", " ")
    return data


global_data = read_data('dataset/글로벌역량/글로벌역량.txt')
active_data = read_data('dataset/능동/능동.txt')
challenge_data = read_data('dataset/도전/용기.txt')
sincerity_data = read_data('dataset/성실/성실.txt')
communication_data = read_data('dataset/소통/소통.txt')
patient_data = read_data('dataset/인내심/인내심.txt')
honesty_data = read_data('dataset/정직/정직.txt')
responsibility_data = read_data('dataset/주인의식/책임.txt')
creative_data = read_data('dataset/창의/창의.txt')
teamwork_data = read_data('dataset/팀워크/팀워크.txt')


In [3]:
pd.options.mode.chained_assignment = None
np.random.seed(0)

# Kolnpy
okt = Okt()

rawdata = [
    global_data,
    active_data,
    challenge_data,
    sincerity_data,
    communication_data,
    patient_data,
    honesty_data,
    responsibility_data,
    creative_data,
    teamwork_data
]

raw_keyword = ['글로벌역량', '능동', '도전', '성실', '소통', '인내심', '정직', '주인의식', '창의', '팀워크']

# raw한 문장데이터를 분석에 알맞게 정제하도록 한다.
def tokenizer(raw, pos=["Noun","Alpha","Verb","Number"], stopword=["하다", "되다"]):
    return [
        word for word, tag in okt.pos(
            raw, 
            norm=True,   # normalize 정제 과정
            stem=True    # stemming 정제 과정
            )
            if len(word) > 1 and tag in pos and word not in stopword
        ]

In [4]:
vectorize = TfidfVectorizer(
    ngram_range=(1,3),
    tokenizer=tokenizer,
    max_df=0.95,
    min_df=0,
    sublinear_tf=True    # tf값에 1+log(tf)를 적용하여 tf값이 무한정 커지는 것을 막음
)
X = vectorize.fit_transform(rawdata)

# X.shape[0]은 문서의 갯수(10개), X.shape[1]는 feature의 갯수(특징되는 단어집합)
print('fit_transform, (sentence {}, feature {})'.format(X.shape[0], X.shape[1]))
 
print(X.toarray())
print(X.shape)
 
# 문장에서 뽑아낸 feature 들의 배열
features = vectorize.get_feature_names()

fit_transform, (sentence 10, feature 111900)
[[0.00923436 0.00923436 0.00545396 ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.01564572 0.01564572 0.01564572]]
(10, 111900)


In [5]:
data_array = X.toarray()
data = pd.DataFrame(data_array, columns=features)
data.shape

(10, 111900)

In [8]:
'''
asarray -> 참조본, 얕은복사 배열
srch_dtm = np.asarray(X.toarray())[:, [vectorize.vocabulary_.get(i) for i in srch]]
srch = search
dtm = Document Term Matrix
'''  

# 검색 문장에서 feature를 뽑아냄
resume_data = read_data('dataset/input_resume.txt')
srch=[t for t in tokenizer(resume_data) if t in features]



# document term matrix 에서 검색하고자 하는 feature만 뽑아낸다.
srch_dtm = np.asarray(X.toarray())[:, [
    # vectorize.vocabulary_.get 는 특정 feature에 가지고 있는 index값을 리턴한다 (features에 단어와 일치하는 것)
    vectorize.vocabulary_.get(i) for i in srch
]]
print(srch_dtm.shape)
print(len(srch)) 

score = srch_dtm.sum(axis=1)
print(score)
# score는 문장별 feature 합계 점수
 
for i in score.argsort()[::-1]:
    if score[i] >= 0:
        print('{} // score : {}'.format(raw_keyword[i], score[i]))


  (0, 81767)	0.00903358573042551
  (0, 88633)	0.008512543083812376
  (0, 17921)	0.017460861532862727
  (0, 92777)	0.006106025163228803
  (0, 30417)	0.0036063168242759986
  (0, 50231)	0.005453962038256885
  (0, 17190)	0.005270434575920099
  (0, 29276)	0.01106373405709583
  (0, 64378)	0.013347672949387194
  (0, 32366)	0.016312527831399554
  (0, 108984)	0.012352196656052779
  (0, 23057)	0.005453962038256885
  (0, 98660)	0.002927840772947771
  (0, 29273)	0.011445751755415273
  (0, 61534)	0.0063441610051313295
  (0, 111210)	0.01069600426130436
  (0, 69086)	0.024686598472721068
  (0, 25993)	0.011445751755415273
  (0, 82057)	0.0055793344122094775
  (0, 83640)	0.008437238928513094
  (0, 109414)	0.004636366006372997
  (0, 30812)	0.0136635733917744
  (0, 15955)	0.021230961700850457
  (0, 48246)	0.005080908924456754
  (0, 907)	0.010123284489213935
  :	:
  (0, 73815)	0.005453962038256885
  (0, 54506)	0.005453962038256885
  (0, 57225)	0.005453962038256885
  (0, 62334)	0.005453962038256885
  (0, 184

In [54]:
import plotly
plotly.__version__

'3.7.1'

In [55]:
import plotly.plotly as py
import plotly.graph_objs as go

In [56]:
bar_data = [go.Bar(
            x=srch,
            y=srch_dtm[1],
            marker=dict(
                color='rgb(158,202,225)',
                line=dict(
                    color='rgb(8,48,107)',
                    width=1.5),
            ),
            opacity=0.6
    )]

plotly.offline.plot(bar_data)

print(srch)
print(len(srch))
# raw_keyword = ['글로벌역량', '능동', '도전', '성실', '소통', '인내심', '정직', '주인의식', '창의', '팀워크']

['여러분', '유영', '주체', '능동', '캐릭터', '쾌감', '선물', '배우', '유영', '대국민', '사기', '여러분', '안방', '극장', '찾아오다', '늦다', '지금', '선택', '집중', '확대', '통한', '능동', '혁신', '도약', '관건', '전망', '화성시', '병점동', '31', '화성시', '능동', '1113-4', '드림', '시티', '106', '31', '가가', '21일', '수원', '지법', '본원', '경매', '앉다', '참으로', '막중', '임무', '무게', '느끼다', '산업혁명', '시대', '능동', '대처', '여건', '특위', '역할', '국립공원', '공단', '북한', '국립공원', '도봉', '사무소', '북한', '둘레길', '능동', '시민', '걷기', '봉사활동', '운영', '북한', '둘레길', '능동', '시민', '걷기', '자원봉사', '음악', '듣다', '박수', '치다', '능동', '참여', '두뇌', '증가', '시키다', '음악', '요법', '능동', '참여', '거나', '수동', '참여', '전두엽', '영향', '배우', '유인나', '능동', '사랑', '매력', '과시', '정면', '마주', '지식', '산실', '이자', '미래', '교육비', '생산', '능동', '연구기관', '정립', '나아가다', '이야기', '김비', '김시현', '맡다', '다음', '시즌', '만들어지다', '능동', '역할', '싶다', '김시현', '열정', '배신', '대통령', '변화', '능동', '대처', '능력', '청년', '갖추다', '바라다', '강조', '사회', '나아가다', '사회', '구성원', '능동', '남녀', '노인', '두루', '이야기', '가구', '세대', '세대', '다른', '유형', '가구', '비다', '능동', '미디어', '소비', '분석', '결과', '부담', '느끼다', '일선', '중간', '관리자

In [50]:
print(features[0:100])

['00초', '00초 기록', '00초 기록 남자', '00초 기록 차지', '010-9307', '010-9307 3810', '010-9307 3810 15일', '010-9307 3810 윤상', '010-9307 3810 책임지다', '010-9307 3810.29', '010-9307 3810.29 명의', '02-401', '02-401 7790', '02-401 7790 능동', '02-446', '02-446 7790', '02-446 7790 유동균', '020560', '020560 한국', '020560 한국 생산', '03월', '03월 19일', '03월 19일 주식', '03월 19일 현재', '069%', '069% 상태', '069% 상태 몰다', '1,678', '1,678 명과', '1,678 명과 유공', '1/3', '1/3 책임지다', '1/3 책임지다 오다', '10', '10 가지', '10 가지 한국', '10 개인', '10 개인 10', '10 구가', '10 구가 운영', '10 대한', '10 대한 조치', '10 대한 처분', '10 대한민국', '10 대한민국 올해', '10 대해', '10 대해 성남시', '10 명의', '10 명의 세무사', '10 방문', '10 방문 개별', '10 보건', '10 보건 기구', '10 보령시', '10 보령시 올해', '10 선정', '10 선정 성실', '10 성남시', '10 성남시 성실', '10 성실', '10 성실 납세', '10 시리즈', '10 시리즈 사용', '10 여명', '10 여명 성희롱', '10 오르다', '10 오르다 중국', '10 오르다 한국', '10 인증', '10 인증 수여', '10 주기', '10 주기 맞다', '10 진로', '10 진로 캠프', '10 진입', '10 진입 모습', '10 짜다', '10 짜다 용기', '10 초등학교', '10 초등학교 활용', '10 출시', '10 출시 밝히다', '10 출시 식품첨가물

In [ ]:
radar_data = [go.Scatterpolar(
  r = score,
  theta = raw_keyword,
  fill = 'toself'
)]

max_value = max(score) + 0.5

layout = go.Layout(
  polar = dict(
    radialaxis = dict(
      visible = True,
      range = [0, max_value]
    )
  ),
  showlegend = False
)

fig = go.Figure(data=radar_data, layout=layout)
plotly.offline.plot(fig)

In [10]:
# scatter 그래프를 그리려는 과정

from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, n_iter=10000, verbose=1)
data_array.shape

(10, 1000)

In [11]:
Z = tsne.fit_transform(data_array.T)
print(Z.shape)

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 1000 samples in 0.003s...
[t-SNE] Computed neighbors for 1000 samples in 0.042s...
[t-SNE] Computed conditional probabilities for sample 1000 / 1000
[t-SNE] Mean sigma: 0.025880
[t-SNE] KL divergence after 250 iterations with early exaggeration: 69.295731
[t-SNE] Error after 2350 iterations: 1.200224
(1000, 2)


In [12]:
import plotly
import plotly.plotly as py
import plotly.graph_objs as go

# Create random data with numpy
import numpy as np


# Create a trace
trace = go.Scatter(
    x = Z[:,0],
    y = Z[:,1],
    mode = 'markers',
    text = features,
    marker = dict(
        size = 6,
        color = '#FFAFC4',
        line = dict(
            width = 1,
            color = '#BF70C4'
        )
    )
)

data_group = [trace]

# Plot and embed in ipython notebook!
plotly.offline.plot(data_group)

'file:///Users/leesunhong/2019capstone/similarity/temp-plot.html'